In [ ]:
# !pyenv local meyf-kaggle-02
# !pyenv shell dmeyf-kaggle-02
# !pyenv versions
# !pyenv virtualenv 3.8.7 dmeyf-kaggle-02
#!pyenv version



In [ ]:
# %%bash 
# pip install duckdb
# pip install jupysql
# pip install duckdb-engine

In [ ]:
# %%bash 
# pip install pandas

In [111]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:////Users/mkiszkurno/Documents/dmeyf/datasets/dmeyf.db


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Y ya podemos usar **SQL** dentro de una notebook!

In [112]:
%%time
%%sql
create or replace table competencia_02 as
select
    *
from read_csv_auto('/Users/mkiszkurno/Documents/dmeyf/datasets/competencia_02_crudo.csv.gz')

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [113]:
%%time
%sql SELECT count(*) FROM COMPETENCIA_02

CPU times: user 107 ms, sys: 29.4 ms, total: 136 ms
Wall time: 311 ms


,count_star()
0,4562810


In [114]:
%%time

%sql SELECT * FROM COMPETENCIA_02 LIMIT 52

CPU times: user 257 ms, sys: 395 ms, total: 652 ms
Wall time: 559 ms


,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,...,8.0,0.00,-35773.32,2.33,5148.0,36602.13,54.0,0.0,5008.71,CONTINUA
1,29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,...,1.0,0.00,-67414.82,1149.57,3299.0,28503.06,10.0,0.0,1395.87,CONTINUA
2,29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,...,1.0,0.00,0.00,0.00,6790.0,5170.25,2.0,0.0,0.00,CONTINUA
3,29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,...,22.0,69653.59,-59422.00,0.00,7037.0,67991.90,31.0,0.0,2897.31,CONTINUA
4,29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,...,8.0,0.00,-4802.21,0.00,7596.0,4851.93,3.0,0.0,492.66,CONTINUA
5,29188277,201901,1,0,1,65,296,1672.70,16113.26,796.59,...,1.0,0.00,-5791.17,0.00,7827.0,12539.39,6.0,0.0,797.64,CONTINUA
6,29188490,201901,1,0,1,43,281,-520.87,6116.69,2138.32,...,8.0,9172.86,-29810.71,0.00,7568.0,73132.65,41.0,0.0,9172.86,CONTINUA
7,29188883,201901,1,0,1,64,296,64862.09,459723.16,988.28,...,8.0,0.00,-1332.76,0.00,7504.0,1102.50,1.0,0.0,58.65,CONTINUA
8,29189950,201901,1,0,1,56,240,13535.41,41250.39,7158.16,...,1.0,0.00,-20419.25,66.25,3717.0,27701.56,19.0,0.0,1137.81,CONTINUA
9,29190599,201901,1,0,1,59,296,2244.16,24670.53,113.76,...,22.0,8727.12,-16718.98,37.05,7137.0,1388.73,2.0,0.0,1219.92,CONTINUA


In [116]:
%%time
%%sql
SELECT foto_mes, 
       COUNT(*) as cantidad 
  FROM competencia_02
 GROUP BY foto_mes

CPU times: user 171 ms, sys: 17.8 ms, total: 189 ms
Wall time: 342 ms


,foto_mes,cantidad
0,201901,124864
1,201902,125898
2,201903,126536
3,201904,127059
4,201905,127683
5,201906,129181
6,201907,130551
7,201908,132615
8,201909,134267
9,201910,136515


In [117]:
%%time
%%sql
SELECT numero_de_cliente, 
       COUNT(*) as cantidad 
  FROM competencia_02
 GROUP BY numero_de_cliente
 having cantidad between 5 and 15
 order by cantidad desc

CPU times: user 2.38 s, sys: 438 ms, total: 2.82 s
Wall time: 608 ms


,numero_de_cliente,cantidad
0,117761277,15
1,118908795,15
2,124757591,15
3,126987247,15
4,133312740,15
...,...,...
27139,131064339,5
27140,137473845,5
27141,145721068,5
27142,153722782,5


In [118]:
%%time
%%sql
create or replace table targets as
with periodos as (
    select distinct foto_mes 
      from competencia_02
), clientes as (
    select distinct numero_de_cliente 
     from competencia_02
), todo as (
    select numero_de_cliente, foto_mes 
     from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , 
        CASE
            WHEN mes_0 + mes_1 + mes_2 = 1 THEN 'BAJA+1'
            WHEN mes_0 + mes_1 + mes_2 = 2 THEN 'BAJA+2'
            WHEN mes_0 + mes_1 + mes_2 = 3 THEN 'CONTINUA'
        END AS clase_ternaria
    from todo t
    left join competencia_02 c using (numero_de_cliente, foto_mes)
) select
  foto_mes
  , numero_de_cliente
  , clase_ternaria,
  mes_0,
  mes_1,
  mes_2
from clase_ternaria where mes_0 = 1

CPU times: user 11.8 s, sys: 2.51 s, total: 14.3 s
Wall time: 2.05 s


,Success


In [119]:
%%time
%%sql 

SELECT * FROM targets T
--where T.clase_ternaria IS NOT NULL 
--where T.numero_de_cliente = 29187499
--where T.numero_de_cliente = 29185587
--where T.numero_de_cliente = 105799389
where T.numero_de_cliente = 118785515
	


CPU times: user 117 ms, sys: 16 ms, total: 133 ms
Wall time: 300 ms


,foto_mes,numero_de_cliente,clase_ternaria,mes_0,mes_1,mes_2
0,201907,118785515,CONTINUA,1,1,1
1,201908,118785515,CONTINUA,1,1,1
2,201909,118785515,CONTINUA,1,1,1
3,201910,118785515,CONTINUA,1,1,1
4,201911,118785515,CONTINUA,1,1,1
5,201912,118785515,CONTINUA,1,1,1
6,202001,118785515,CONTINUA,1,1,1
7,202002,118785515,CONTINUA,1,1,1
8,202003,118785515,CONTINUA,1,1,1
9,202004,118785515,CONTINUA,1,1,1


In [120]:
%%time
%%sql
alter table competencia_02 add column clase_ternaria VARCHAR(10)

RuntimeError: (duckdb.duckdb.CatalogException) Catalog Error: Column with name clase_ternaria already exists!
[SQL: alter table competencia_02 add column clase_ternaria VARCHAR(10)]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [121]:
%%time
%sql SELECT * FROM COMPETENCIA_02 LIMIT 15

CPU times: user 237 ms, sys: 55.7 ms, total: 293 ms
Wall time: 330 ms


,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,...,8.0,0.00,-35773.32,2.33,5148.0,36602.13,54.0,0.0,5008.71,CONTINUA
1,29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,...,1.0,0.00,-67414.82,1149.57,3299.0,28503.06,10.0,0.0,1395.87,CONTINUA
2,29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,...,1.0,0.00,0.00,0.00,6790.0,5170.25,2.0,0.0,0.00,CONTINUA
3,29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,...,22.0,69653.59,-59422.00,0.00,7037.0,67991.90,31.0,0.0,2897.31,CONTINUA
4,29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,...,8.0,0.00,-4802.21,0.00,7596.0,4851.93,3.0,0.0,492.66,CONTINUA
5,29188277,201901,1,0,1,65,296,1672.70,16113.26,796.59,...,1.0,0.00,-5791.17,0.00,7827.0,12539.39,6.0,0.0,797.64,CONTINUA
6,29188490,201901,1,0,1,43,281,-520.87,6116.69,2138.32,...,8.0,9172.86,-29810.71,0.00,7568.0,73132.65,41.0,0.0,9172.86,CONTINUA
7,29188883,201901,1,0,1,64,296,64862.09,459723.16,988.28,...,8.0,0.00,-1332.76,0.00,7504.0,1102.50,1.0,0.0,58.65,CONTINUA
8,29189950,201901,1,0,1,56,240,13535.41,41250.39,7158.16,...,1.0,0.00,-20419.25,66.25,3717.0,27701.56,19.0,0.0,1137.81,CONTINUA
9,29190599,201901,1,0,1,59,296,2244.16,24670.53,113.76,...,22.0,8727.12,-16718.98,37.05,7137.0,1388.73,2.0,0.0,1219.92,CONTINUA


In [122]:
%%time
%%sql

update competencia_02
   set clase_ternaria = targets.clase_ternaria
  from targets
 where competencia_02.numero_de_cliente = targets.numero_de_cliente
   and competencia_02.foto_mes = targets.foto_mes

CPU times: user 3.23 s, sys: 517 ms, total: 3.74 s
Wall time: 2.09 s


,Success


In [123]:
%%sql
select numero_de_cliente, foto_mes, clase_ternaria from competencia_02 
--where numero_de_cliente = 29187499
where numero_de_cliente = 29185587
--where numero_de_cliente = 105799389
--where numero_de_cliente = 118785515
 

,numero_de_cliente,foto_mes,clase_ternaria
0,29185587,201901,CONTINUA
1,29185587,201902,CONTINUA
2,29185587,201903,CONTINUA
3,29185587,201904,CONTINUA
4,29185587,201905,CONTINUA
5,29185587,201906,CONTINUA
6,29185587,201907,CONTINUA
7,29185587,201908,CONTINUA
8,29185587,201909,CONTINUA
9,29185587,201910,CONTINUA


In [124]:
%%time
%%sql
copy competencia_02 to '/Users/mkiszkurno/Documents/dmeyf/datasets/competencia_02.csv.gz' (FORMAT CSV, HEADER)

CPU times: user 6min 26s, sys: 15.3 s, total: 6min 41s
Wall time: 2min 58s


,Success


In [ ]:
%%time
%%sql
create or replace table competencia_02_verificar as
select
    *
from read_csv_auto('/Users/mkiszkurno/Documents/dmeyf/datasets/competencia_02.csv.gz')

In [ ]:
%%time
%sql SELECT count(*) FROM competencia_02_verificar